In [23]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [24]:
if not os.path.exists('temp'):
    os.mkdir('temp')

In [25]:
from langchain.tools import tool
from typing import List, Dict
from langchain_teddynote.tools import GoogleNews
from langchain_community.agent_toolkits import FileManagementToolkit

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_teddynote.messages import AgentStreamParser

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [26]:
@tool
def lastest_news(k: int = 5) -> List[Dict[str, str]]:
    """Look up latest news"""

    news_tool = GoogleNews()
    
    return news_tool.search_latest(k=k) 

In [27]:
answer = lastest_news.invoke({'k': 2})

for ans in answer:
    print(ans)

{'url': 'https://news.google.com/rss/articles/CBMiakFVX3lxTFBlSENJVkpsQWNyMnI4bFdabUJ6SDdHSWFtNXVlSmdHOENENVhKazB5YzhfdVhyZFJ4TFZ2OUpYZHZFaXVIYzVIdVRwVWJvUXlMZXJyVWY3U3dNRGxSWEFwdEhUWlhoVmRVWEE?oc=5', 'content': '[속보] 이준석 “후보 단일화, 이번 선거에 없다…김문수로 못 이겨” - 한겨레'}
{'url': 'https://news.google.com/rss/articles/CBMiVkFVX3lxTE1QRFpYMmlWSDVjTmozREo5cVV1dlBab2hVWWswR1lNdHV0cXZIWkdpbXdNMG5PV2gyTEMyc0h4aUpUcDkxbEhvMURiQlJVdlRXcURGRW5R?oc=5', 'content': '[속보] 경찰, ‘내란 혐의’ 한덕수·최상목 출국금지 - 매일경제'}


In [28]:
working_directory = 'temp'

toolkit = FileManagementToolkit(root_dir=str(working_directory))
tools = toolkit.get_tools()

In [29]:
tools

[CopyFileTool(root_dir='temp'),
 DeleteFileTool(root_dir='temp'),
 FileSearchTool(root_dir='temp'),
 MoveFileTool(root_dir='temp'),
 ReadFileTool(root_dir='temp'),
 WriteFileTool(root_dir='temp'),
 ListDirectoryTool(root_dir='temp')]

In [30]:
tools.append(lastest_news)

In [31]:
tools

[CopyFileTool(root_dir='temp'),
 DeleteFileTool(root_dir='temp'),
 FileSearchTool(root_dir='temp'),
 MoveFileTool(root_dir='temp'),
 ReadFileTool(root_dir='temp'),
 WriteFileTool(root_dir='temp'),
 ListDirectoryTool(root_dir='temp'),
 StructuredTool(name='lastest_news', description='Look up latest news', args_schema=<class 'langchain_core.utils.pydantic.lastest_news'>, func=<function lastest_news at 0x00000263471ECAE0>)]

In [32]:
for tool in tools:
    print(f'{tool.name} : {tool.description}')

copy_file : Create a copy of a file in a specified location
file_delete : Delete a file
file_search : Recursively search for files in a subdirectory that match the regex pattern
move_file : Move or rename a file from one location to another
read_file : Read file from disk
write_file : Write file to disk
list_directory : List files and directories in a specified folder
lastest_news : Look up latest news


In [33]:
llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini'
)

In [34]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system', 
            'You are a helpful assistant. '
            'Make sure to use the `lastest_news` tool to find lastest news. '
            'Make sure to use the `file_managent` tool to manage files. '
        ),
        ('placeholder', '{chat_history}'),
        ('human', '{input}'),
        ('placeholder', '{agent_scratchpad}')
    ]
)

In [35]:
agent = create_tool_calling_agent(llm, tools, prompt)      

In [36]:
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=False, 
    max_iterations=10, 
    max_execution_time=10, 
    handle_parsing_errors=True
)

In [37]:
store = {}

def get_session_history(session_ids):  # session_ids ->'abc123'
    if session_ids not in store:
        store[session_ids] = ChatMessageHistory()
    
    return store[session_ids]

In [38]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor, # 에이전트 실행기
    get_session_history, # 대화내용 기록 메모리 함수
    input_messages_key='input',  # 사용자 질문
    history_messages_key='chat_history'
)

In [39]:
agent_stream_parser = AgentStreamParser()

In [40]:
result = agent_with_chat_history.stream(
    {
        'input': '최신 뉴스 5개를 검색하고, 각 뉴스의 제목을 파일명으로 하는 파일을 생성하고(.txt), '
        '파일의 뉴스 내용은 뉴스의 내용과 url을 추가하세요.'
    },
    config={'configurable': {'session_id': 'abc123'}}
)

In [41]:
for step in result:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: lastest_news
k: 5
Log: 
Invoking: `lastest_news` with `{'k': 5}`



[관찰 내용]
Observation: [{'url': 'https://news.google.com/rss/articles/CBMiakFVX3lxTFBlSENJVkpsQWNyMnI4bFdabUJ6SDdHSWFtNXVlSmdHOENENVhKazB5YzhfdVhyZFJ4TFZ2OUpYZHZFaXVIYzVIdVRwVWJvUXlMZXJyVWY3U3dNRGxSWEFwdEhUWlhoVmRVWEE?oc=5', 'content': '[속보] 이준석 “후보 단일화, 이번 선거에 없다…김문수로 못 이겨” - 한겨레'}, {'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTE53dW1WblQ4ZlZVMnZvbzA0X2RRUl9QMG9HVTZzNkVEMnN3dTVBSTZTQTJaSlJKZTNpNUg3WGl2NnRteU5RenFtbXBpMGU4dFNoOGx4d2tvcGhLZ9IBX0FVX3lxTE9ZdnAwbjZTVFBmZlE2TjBHaDl3dVNVdG1wbFpTd19HNjUzZDlNSjJPalNDZDhKNGl4TjV2X3YyWnBTUl9MTk9vbUlzNzRtSTJ3aVdka2F0TWI1bGFrTU1R?oc=5', 'content': '[여론조사 ‘경향’] 이재명 하락폭보다 김문수 상승폭이 더 높았다···지지율 격차 더 좁혀져 - 경향신문'}, {'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTFBYMmFRako1R0kxLUtvMHkwMGZ4U3gtbkNublloOXI2bUhaUjBrX0NyMEF4UG1aeWdadXZnU2VRbW5GeGFXV2VuUWNTMkplUHJHUGdOVUY4Rmc2UQ?oc=5', 'content': '"김문수 보고 나도 생각 나" 박근혜, 박정희 생가 방문 - 한국경제'}, {'url': 'https://

Stopping agent prematurely due to triggering stop condition


[도구 호출]
Tool: write_file
file_path: 이준석_후보_단일화_이번_선거에_없다.txt
text: [속보] 이준석 “후보 단일화, 이번 선거에 없다…김문수로 못 이겨”
URL: https://news.google.com/rss/articles/CBMiakFVX3lxTFBlSENJVkpsQWNyMnI4bFdabUJ6SDdHSWFtNXVlSmdHOENENVhKazB5YzhfdVhyZFJ4TFZ2OUpYZHZFaXVIYzVIdVRwVWJvUXlMZXJyVWY3U3dNRGxSWEFwdEhUWlhoVmRVWEE?oc=5
Log: 
Invoking: `write_file` with `{'file_path': '이준석_후보_단일화_이번_선거에_없다.txt', 'text': '[속보] 이준석 “후보 단일화, 이번 선거에 없다…김문수로 못 이겨”\nURL: https://news.google.com/rss/articles/CBMiakFVX3lxTFBlSENJVkpsQWNyMnI4bFdabUJ6SDdHSWFtNXVlSmdHOENENVhKazB5YzhfdVhyZFJ4TFZ2OUpYZHZFaXVIYzVIdVRwVWJvUXlMZXJyVWY3U3dNRGxSWEFwdEhUWlhoVmRVWEE?oc=5'}`



[도구 호출]
Tool: write_file
file_path: 이재명_하락폭보다_김문수_상승폭이_더높았다.txt
text: [여론조사 ‘경향’] 이재명 하락폭보다 김문수 상승폭이 더 높았다···지지율 격차 더 좁혀져
URL: https://news.google.com/rss/articles/CBMiWkFVX3lxTE53dW1WblQ4ZlZVMnZvbzA0X2RRUl9QMG9HVTZzNkVEMnN3dTVBSTZTQTJaSlJKZTNpNUg3WGl2NnRteU5RenFtbXBpMGU4dFNoOGx4d2tvcGhLZ9IBX0FVX3lxTE9ZdnAwbjZTVFBmZlE2TjBHaDl3dVNVdG1wbFpTd19HNjUzZDlNSjJPalNDZDhKNGl4TjV2

In [42]:
result = agent_with_chat_history.stream(
    {
        'input': '좀전에 했던 질문이 뭐였지?'
    },
    config={'configurable': {'session_id': 'abc123'}}
)

for step in result:
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
당신은 최신 뉴스 5개를 검색하고, 각 뉴스의 제목을 파일명으로 하는 텍스트 파일을 생성해 달라고 요청하셨습니다. 파일에는 뉴스의 내용과 URL을 추가해야 했습니다.
